In [1]:
#!/usr/bin/env python 
# -*- coding: utf-8 -*-



##---------------------------------------------------------------------------------------
#
#        UTILS functions
#
##---------------------------------------------------------------------------------------


# we get the xml file in this directory
def getXmlFile(flibasedir):
    import os
    dirs = os.listdir( flibasedir )
    #print dirs

    import re
    xmlmatch = r'(.*)xml'
    for dir in dirs:
        xmlfound=re.search(xmlmatch,dir,flags=0)
        if xmlfound:
            #print xmlfound.group()
            return flibasedir + '/' + xmlfound.group()



def getEpiInfos(xmlfile):
    result = {}
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        # epibold directory
        for epibold in root.iter('EPIBOLD'):
            for child in epibold:
                print child.text
                if child.tag == 'file':
                    epidir = child.text
                    result['epidir'] = epidir
                    print epidir
                    
                # parameters
                if child.tag == 'parameters':
                    print "parameters"
                    
                    for child2 in child:
                                                     
                        # TR
                        if child2.tag == 'TR':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    tr = child3.text
                                    print tr
                                    result['TR'] = tr
                    
                        # dynamics
                        if child2.tag == 'dynamics':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    dynamics = child3.text
                                    result['dynamics'] = dynamics
                                    
                        # sliceTimingVector
                        if child2.tag == 'sliceTimingVector':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    sliceTimingVector = child3.text
                                    result['sliceTimingVector'] = sliceTimingVector
                                    
                        # nb_slices
                        if child2.tag == 'nb_slices':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    nb_slices = child3.text
                                    result['nb_slices'] = nb_slices
                
        
    except:
        print 'exception'
    return result



def getT1file(xmlfile):
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        # T1 directory and file
        for t1 in root.iter('T1'):
            for child in t1:
                #print child.text
                if child.tag == 'file':
                    t1 = child.text
                    t1 = flibasedir + t1
                    # print  t1
    except:
        print 'exception'
    return t1
    
    



##---------------------------------------------------------------------------------------
#
#    end    UTILS functions
#
##---------------------------------------------------------------------------------------

##---------------------------------------------------------------------------------------
#
#    Function PREPROCESS 
#
##---------------------------------------------------------------------------------------

def preprocess(spm_standalone, mcr, flibasedir,atlasfile, resultdir):


# ## preliminaries



    import nipype.interfaces.io as nio           # Data i/o
    import nipype.interfaces.spm as spm          # spm
    import nipype.interfaces.matlab as mlab      # how to run matlab
    import nipype.interfaces.utility as util     # utility
    import nipype.pipeline.engine as pe          # pypeline engine
    import nipype.algorithms.rapidart as ra      # artifact detection
    import nipype.algorithms.modelgen as model   # model specification
    import os                                    # system functions


    

    from nipype.interfaces import spm

    # essai du mercredi soir matlab_cmd = ' '.join([spm_standalone,mcr,'batch','script'])
    matlab_cmd = ' '.join([spm_standalone,mcr,'batch'])
    spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)


    # ## Nipype verbosity


    import nipype
    # Optional: Use the following lines to increase verbosity of output
    nipype.config.set('logging', 'workflow_level',  'INFO')
    nipype.config.set('logging', 'interface_level', 'INFO')
    nipype.logging.update_logging(nipype.config)
    
    ## Nipype provenance marche pas
    #nipype.config.set('provenance', 'True')
    from nipype import config
    #config.enable_provenance()


# ## 1 - we decode the xml file


    xmlfile = getXmlFile(flibasedir)

    print xmlfile


    # In[8]:

    # the we decode the xmlfile
    import xml.etree.ElementTree as ET
    tree = ET.parse(xmlfile)
    root = tree.getroot()


    print root.tag


    epiResults = getEpiInfos(xmlfile)
    print epiResults



    t1 = getT1file(xmlfile)
    print t1


    # # Start Pileline -> declaration


    preproc = pe.Workflow(name='preproc')

    # level1.base_dir = os.path.abspath('spm_auditory_tutorial/workingdir')
    preproc.base_dir = os.path.abspath('/scratch/user/hirsch/datadir4/data_results_py/_report')
    
    # ## 1 - first node data grabbing by select files 

    #os.path.join(dir_name, base_filename + suffix)

    epidir = epiResults['epidir']
    
    abs_epidir = os.path.join(flibasedir, epidir)
    abs_t1 = os.path.join(flibasedir, t1)

    from nipype import SelectFiles, Node
    templates = dict(T1=abs_t1,
                     epi= abs_epidir + "/" + "*.nii")

    filesource = Node(SelectFiles(templates), "filesource")
    filesource.inputs.subject_id = "subj1"
    filesource.outputs.get()


    # ## 2 - segment T1


    segment = pe.Node(interface=spm.NewSegment(), name='segment')

    #seg.inputs.channel_files = 't0009_t1_s03.nii'
    segment.inputs.channel_info =(0.001,60.0,(True,True))          
    segment.inputs.sampling_distance= 3.0
    # todo donner un chemin de TPM.nii comme argument de la future fct
    tissue1 = ((atlasfile , 1), 1, (True,False), (False, False))
    tissue2 = ((atlasfile, 2), 1, (True,False), (False, False))
    tissue3 = ((atlasfile, 3), 2, (True,False), (False, False))
    tissue4 = ((atlasfile, 4), 3, (True,False), (False, False))
    tissue5 = ((atlasfile, 5), 4, (True,False), (False, False))
    tissue6 = ((atlasfile, 6), 2, (False,False), (False, False))
    segment.inputs.tissues = [tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
    segment.inputs.affine_regularization = 'mni'
    #segment.inputs.warping_regularization = [0.0, 0.001, 0.5, 0.05, 0.2]
    segment.inputs.write_deformation_fields = [False, False]

    # 2 - segment
    preproc.connect(filesource,"T1" ,segment, 'channel_files')
    

    
    # marche pas
    #segment.config['execution']['create_report']= 'True'
    #segment.write_report(report_type='postexec', cwd='/scratch/user/hirsch/datadir4/data_results_py_test_spm/cmd_reports')
    
    preproc.run()
    
    print segment.output_dir()
    rapport = segment.output_dir() + '/_report/report.rst'
    
    from shutil import copyfile

    copyfile(rapport, '/scratch/user/hirsch/datadir4/data_results_py/_report/report_segment.rst')
    
    return 0


In [2]:
preprocess('/homes_unix/hirsch/historique_fli_iam/essai_spm_stand_alone/spm12/run_spm12.sh' ,'/homes_unix/hirsch/historique_fli_iam/essai_spm_stand_alone/mcr2016/v91', '/scratch/user/hirsch/datadir/data_set/t0009/repos01', '/scratch/user/hirsch/datadir/data_set/t0009/repos01/Atlases/TPM.nii' , '/scratch/user/hirsch/datadir3/data_results_py' )

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node filesource in dir: /scratch/user/hirsch/datadir4/data_results_py/_report/preproc/filesource
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node segment in dir: /scratch/user/hirsch/datadir4/data_results_py/_report/preproc/segment
INFO:workflow:Runtime memory and threads stats unavailable


/scratch/user/hirsch/datadir/data_set/t0009/repos01/xml_example.xml
RS_analysis
EPIBOLD
EPIBOLD
bold-dataset

     
parameters
2
{'sliceTimingVector': '1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 2 4 6 8 10 12 14 16 18 20 22 24 26 28 30', 'dynamics': '240', 'nb_slices': '31', 'TR': '2', 'epidir': 'EPIBOLD'}
exception
T1/t0009_t1_s03.nii
/tmp/tmpz9Tob5/preproc/segment


IOError: [Errno 2] No such file or directory: '/tmp/tmpz9Tob5/preproc/segment/_report/report.rst'

In [ ]:
from nipype import config
config.enable_provenance()

In [ ]:
import nipype.interfaces.spm as spm
spm.NewSegment().help()